# Setup: Mount Drive, Install Dependencies & Import Libraries

In [ ]:
#Setup: Mount Drive, Install Dependencies & Import Libraries (One Per Line)** { display-mode: "form" }
#@markdown
from google.colab import drive
drive.mount('/content/drive')

!apt update -y
!apt install -y ffmpeg
!apt install -y aria2
!apt install -y wget

!pip install -q -U yt-dlp
!pip install -q -U ipywidgets
!pip install -q -U requests
!pip install -q -U beautifulsoup4
!pip install -q -U python-magic
!pip install -q -U m3u8downloader

import os
import subprocess
import shlex
import re
import shutil
import time
import json
import mimetypes
import magic
import requests
from yt_dlp import YoutubeDL
from IPython.display import display
from IPython.display import HTML
from ipywidgets import IntProgress
from zipfile import ZipFile
from bs4 import BeautifulSoup
from urllib.parse import urlparse
from urllib.parse import unquote


# Compress Video

In [ ]:
# Compress Video (with Progress Bar)** { display-mode: "form" }
video_path = "/content/input.mp4"  #@param {type:"string"}
output_path = "/content/output_compressed.mp4"  #@param {type:"string"}
crf = "23"  #@param {type:"string"}

if not os.path.exists(video_path):
    print(f"🔴 Error: Input video path does not exist: {video_path}")
else:
    cmd_probe = f'ffprobe -v error -show_entries format=duration -of default=noprint_wrappers=1:nokey=1 "{video_path}"'
    try:
        duration = float(subprocess.check_output(shlex.split(cmd_probe)))
        bar = IntProgress(min=0, max=100, description="Compressing:")
        display(bar)

        output_dir_compress = os.path.dirname(output_path)
        if output_dir_compress and not os.path.exists(output_dir_compress):
            os.makedirs(output_dir_compress, exist_ok=True)

        cmd = f'ffmpeg -i "{video_path}" -vcodec libx264 -crf {crf} "{output_path}" -progress pipe:1 -nostats -y'
        print(f"Executing: {cmd}")
        proc = subprocess.Popen(shlex.split(cmd), stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, encoding='utf-8', errors='replace')

        for line in proc.stdout:
            # print(line, end='') # Uncomment for detailed ffmpeg output
            m = re.search(r'out_time_ms=(\d+)', line)
            if m:
                t = int(m.group(1)) / 1_000_000.0
                if duration > 0:
                    pct = min(100, int((t / duration) * 100))
                    bar.value = pct

        proc.wait()
        if proc.returncode == 0:
            bar.value = 100
            print("\n✅ Compression complete.")
        else:
            bar.bar_style = 'danger'
            bar.value = 0 # Reset or indicate failure
            print(f"\n🔴 Compression failed. FFmpeg Error Code: {proc.returncode}")
            # To see full error, you might need to capture stderr separately if not merged into stdout
    except FileNotFoundError:
        print("🔴 Error: ffprobe or ffmpeg not found. Ensure they are installed and in PATH.")
    except ValueError:
        print("🔴 Error: Could not determine video duration. Is ffprobe working correctly and is the video file valid?")
    except Exception as e:
        print(f"🔴 An unexpected error occurred during compression: {e}")

# File Manager

In [ ]:
# File Manager (with Progress Bar)** { display-mode: "form" }
operation = "move"  #@param ["move","delete","copy"] {type:"string"}
source_path = "/content/drive/MyDrive/source_folder"  #@param {type:"string"}
dest_path = "/content/drive/MyDrive/destination_folder"  #@param {type:"string"}

files_to_process = []
if not os.path.exists(source_path):
    print(f"🔴 Error: Source path does not exist: {source_path}")
elif operation in ["move", "copy"] and not dest_path:
    print(f"🔴 Error: Destination path is required for {operation} operation.")
else:
    if operation == "move" or operation == "copy":
        for root, dirs, fs in os.walk(source_path):
            for f_name in fs:
                src_file = os.path.join(root, f_name)
                # Create the destination path by appending the relative path from source_path to dest_path
                relative_path_to_file = os.path.relpath(src_file, source_path)
                dst_file = os.path.join(dest_path, relative_path_to_file)
                files_to_process.append((src_file, dst_file))
    elif operation == "delete":
        for root, dirs, fs in os.walk(source_path, topdown=False): # topdown=False for deleting files first
            for f_name in fs:
                files_to_process.append(os.path.join(root, f_name))
            for d_name in dirs:
                 files_to_process.append(os.path.join(root, d_name)) # Add directories to delete after their contents
        if os.path.isdir(source_path): # Ensure the source_path itself is added if it's a directory and delete is chosen
             files_to_process.append(source_path)

    if not files_to_process and operation != "delete": # For delete, source_path itself might be the only item if empty
        print(f"No files found in source path: {source_path} to {operation}")
    else:
        bar = IntProgress(min=0, max=len(files_to_process) if files_to_process else 1, description=operation.title()+":")
        display(bar)
        processed_count = 0
        for idx, item in enumerate(files_to_process, 1):
            try:
                if operation == "move":
                    src, dst = item
                    os.makedirs(os.path.dirname(dst), exist_ok=True)
                    shutil.move(src, dst)
                elif operation == "copy":
                    src, dst = item
                    os.makedirs(os.path.dirname(dst), exist_ok=True)
                    shutil.copy2(src, dst) # copy2 preserves metadata
                elif operation == "delete":
                    path_to_delete = item
                    if os.path.isdir(path_to_delete):
                        shutil.rmtree(path_to_delete, ignore_errors=False) # ignore_errors=False to catch issues
                    elif os.path.isfile(path_to_delete):
                        os.remove(path_to_delete)
                processed_count +=1
            except Exception as e:
                print(f"\n🔴 Error {operation}ing {item}: {e}")
            bar.value = idx

        bar.value = bar.max # Ensure bar is full on completion
        print(f"\n✅ {operation.title()} complete. Processed {processed_count} items.")

# Universal Downloader

In [ ]:
# Run to Download
# Description: Define the URL, download type, and output path here.
# Error handling for the output path is included.
# Input Parameters
url = "https://kebab.bunkr.ru/1e02c98e-c67a-4409-85e7-1bad235eb445.mp4" #@param {type:"string"}
download_type = "d" #@param ["a", "b", "c", "d"] {allow-input: true}
output_path = "/content/drive/MyDrive/Downloads" #@param {type:"string"}

# Error handling for output_path
if not output_path:
    print("Error: Output path cannot be empty. Using default: /content/downloads")
    output_path = "/content/downloads"

if not os.path.exists(output_path):
    try:
        os.makedirs(output_path, exist_ok=True)
        print(f"Output directory created: {output_path}")
    except OSError as e:
        print(f"Error creating output directory {output_path}: {e}. Please check the path and permissions.")
        # Optionally, you might want to stop execution or use a fallback
        output_path = "/content/downloads" # Fallback to a default if creation fails
        if not os.path.exists(output_path):
             os.makedirs(output_path, exist_ok=True) # Try creating default
        print(f"Using fallback output directory: {output_path}")
elif not os.path.isdir(output_path):
    print(f"Error: The specified output path '{output_path}' is a file, not a directory. Please provide a valid directory path.")
    # Optionally, you might want to stop execution or use a fallback
    output_path = "/content/downloads" # Fallback
    if not os.path.exists(output_path):
         os.makedirs(output_path, exist_ok=True)
    print(f"Using fallback output directory: {output_path}")
else:
    print(f"Using output directory: {output_path}")

# Further ensure the output path ends with a separator if it's a directory for file concatenation
if not output_path.endswith(os.sep):
    output_path += os.sep

print(f"URL: {url}")
print(f"Download Type: {download_type}")
print(f"Output Path: {output_path}")



In [ ]:
#@markdown <font size=2.5>Run Cell to Start Download
# Description: This cell contains the consolidated download functions and
# executes the download based on the 'download_type' from Cell 2.

# --- Consolidated Download Functions ---

# Function a: wget Downloader
def download_with_wget(file_url, save_location):
    print(f"Starting wget download for: {file_url}")
    try:
        # Ensure the save_location is a directory
        if not os.path.isdir(save_location):
             # Attempt to create if it doesn't exist, or raise error
            os.makedirs(save_location, exist_ok=True)
            print(f"Created directory for wget: {save_location}")

        command = f'wget -P "{save_location}" "{file_url}"'
        process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        stdout, stderr = process.communicate()

        if process.returncode == 0:
            print("wget download successful!")
            # Try to find the downloaded filename (wget doesn't easily return it)
            # This is a basic way; more robust parsing of stdout might be needed
            lines = stdout.decode().splitlines() + stderr.decode().splitlines()
            saved_file_line = next((line for line in lines if " saved [" in line or " -> " in line or "Saving to: ‘" in line), None) #
            if saved_file_line:
                # Extract filename, this is highly dependent on wget output format
                match = re.search(r"(?:Saving to: ‘|-> ‘)([^’]+)", saved_file_line) #
                if match:
                    filename = os.path.basename(match.group(1).strip('‘’'))
                    print(f"File saved as: {os.path.join(save_location, filename)}")
                else:
                    print(f"File saved in: {save_location}. Could not determine exact filename from output.")
            else:
                print(f"File saved in: {save_location}. Could not determine exact filename from output.")

        else:
            print("wget download failed.")
            print(f"Stderr: {stderr.decode()}")
            print(f"Stdout: {stdout.decode()}")
    except Exception as e:
        print(f"An error occurred during wget download: {e}")

# --- Functions for Bunkr Downloader (Type 'b') ---
def extract_id_from_url_bunkr(url): #
    album_pattern = r'https?://(?:bunkr|bunkrr)\.(?:is|cr|ru|to|la|se)/a/(\w+)' #
    album_match = re.match(album_pattern, url) #
    if album_match: #
        return {'type': 'album', 'id': album_match.group(1)} #

    file_pattern = r'https?://(?:bunkr|bunkrr)\.(?:is|cr|ru|to|la|se)/(?:[dfvi])/([^/]+)' #
    file_match = re.match(file_pattern, url) #
    if file_match: #
        return {'type': 'file', 'id': file_match.group(1)} #

    cache_pattern = r'https?://(?:soup\.bunkr\.(?:se|is|cr|ru|to|la)|c\.bunkr-cache\.se|cdn[0-9]*\.bunkr\.(?:is|cr|ru|to|la))/([^/]+)/([^?]+)' #
    cache_match = re.match(cache_pattern, url) #
    if cache_match: #
        return {'type': 'direct', 'id': cache_match.group(2)} #

    raise ValueError("Invalid Bunkr URL format. Expected formats:\n- Album: https://bunkr.ru/a/ALBUMID\n- File: https://bunkr.ru/[d|f|v|i]/FILEID\n- Direct: https://c.bunkr-cache.se/*/FILENAME or https://soup.bunkr.ru/*/FILENAME") #

def verify_video_content_bunkr(file_path): #
    try:
        mime = magic.Magic(mime=True) #
        file_type = mime.from_file(file_path) #
        if file_type.startswith('video/'): #
            print(f"✅ Verified as valid video file: {file_type}") #
            return True #
        elif file_type in ['text/html', 'image/png', 'image/jpeg', 'image/gif']: #
            print(f"❌ Download failed - received {file_type} instead of video") #
            return False #
        else:
            file_size = os.path.getsize(file_path) / (1024 * 1024)  # Size in MB #
            if file_size < 0.5: #
                print(f"❌ File too small ({file_size:.2f} MB) - likely not a video") #
                return False #
            else:
                print(f"⚠️ Unrecognized file type: {file_type}, but size seems appropriate ({file_size:.2f} MB)") #
                return True #
    except Exception as e: #
        print(f"Error verifying file content: {e}") #
        return False #

def get_direct_download_url_bunkr(viewer_url, headers): #
    try:
        print(f"Processing viewer page: {viewer_url}") #
        user_agents = [ #
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36', #
            'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.0 Safari/605.1.15', #
            'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36' #
        ]
        download_url = None #
        for agent in user_agents: #
            headers['User-Agent'] = agent #
            response = requests.get(viewer_url, headers=headers) #
            if response.status_code != 200: #
                print(f"Failed to access file viewer with agent {agent}: Status code {response.status_code}") #
                continue #
            soup = BeautifulSoup(response.text, 'html.parser') #
            if "geo restriction" in response.text.lower() or "content unavailable" in response.text.lower(): #
                 print("⚠️ Detected geo-restriction or content unavailable message") #
                 continue #
            for script in soup.find_all('script'): #
                if script.string: #
                    url_match = re.search(r'(?:streamUrl|downloadUrl|mediaUrl|src)["\']\s*:\s*["\']([^"\']+)["\']', script.string) #
                    if url_match: #
                        download_url = url_match.group(1).replace('\\', '') #
                        break #
                    file_match = re.search(r'(https?://[^"\']*(?:\.mp4|\.webm|\.mov|\.avi|\.mkv))', script.string) #
                    if file_match: #
                        download_url = file_match.group(1) #
                        break #
            if not download_url: #
                media_elem = soup.find('video') #
                if media_elem: #
                    source = media_elem.find('source') #
                    download_url = source.get('src') if source else media_elem.get('src') #
            if not download_url: #
                download_btn = soup.select_one('a.download-btn, a[download], a.download-link') #
                if download_btn: #
                    download_url = download_btn.get('href') #
            if download_url: #
                if not download_url.startswith('http'): #
                    base_url = urlparse(viewer_url) #
                    domain = f"{base_url.scheme}://{base_url.netloc}" #
                    download_url = f"{domain}{download_url}" #
                print(f"Extracted direct download URL: {download_url}") #
                if not any(ext in download_url.lower() for ext in ['.mp4', '.webm', '.mov', '.avi', '.mkv']): #
                    print("⚠️ URL doesn't appear to be a video file. Will verify after download.") #
                return download_url #
        if not download_url: #
            print("Could not find direct download URL - likely geo-restricted or content unavailable") #
            return None #
    except Exception as e: #
        print(f"Failed to process viewer page: {e}") #
        return None #

def extract_filename_from_url_bunkr(url): #
    parsed_url = urlparse(url) #
    path = parsed_url.path #
    filename = os.path.basename(path) #
    if parsed_url.query: #
        query_params = parsed_url.query.split('&') #
        for param in query_params: #
            if param.startswith('n='): #
                query_filename = param[2:] #
                if query_filename: #
                    filename = unquote(query_filename) #
                    break #
    if not any(filename.lower().endswith(ext) for ext in ['.mp4', '.webm', '.mov', '.avi', '.mkv']): #
        if '.mp4' in url.lower(): #
            filename += '.mp4' #
        elif '.webm' in url.lower(): #
            filename += '.webm' #
        elif '.mov' in url.lower(): #
            filename += '.mov' #
        else: #
            filename += '.mp4' #
    return unquote(filename) #

def download_file_bunkr(file_url, file_name, download_path, headers): #
    try:
        try:
            head_response = requests.head(file_url, headers=headers, timeout=10) #
            file_size = int(head_response.headers.get('content-length', 0)) / (1024 * 1024) #
            content_type = head_response.headers.get('content-type', '') #
            if content_type and 'video' not in content_type and 'octet-stream' not in content_type: #
                print(f"⚠️ Content-Type '{content_type}' doesn't appear to be video. Will verify after download.") #
            print(f"File size: {file_size:.2f} MB") #
            if file_size < 0.5: #
                print("⚠️ File seems too small for a video. Proceeding anyway but will verify.") #
        except Exception as e: #
            print(f"Could not determine file information: {e}") #
        file_name = re.sub(r'[\\/*?:"<>|]', "_", file_name) #
        file_path = os.path.join(download_path, file_name) #
        successful = False #
        retry_count = 0 #
        while not successful and retry_count < 3: #
            try:
                response = requests.get(file_url, headers=headers, stream=True, timeout=30) #
                if response.status_code != 200: #
                    print(f"Failed to download {file_name}: Status code {response.status_code}") #
                    retry_count += 1 #
                    time.sleep(2) #
                    continue #
                with open(file_path, 'wb') as f: #
                    total_length = int(response.headers.get('content-length', 0)) #
                    dl = 0 #
                    for chunk in response.iter_content(chunk_size=8192): #
                        if chunk: #
                            dl += len(chunk) #
                            f.write(chunk) #
                            done = int(50 * dl / total_length) if total_length > 0 else 0 #
                            print(f"\r[{'=' * done}{' ' * (50-done)}] {dl*100/total_length:.2f}%" if total_length > 0 else f"\r{dl/1024/1024:.2f} MB downloaded", end='') #
                    print() #
                if verify_video_content_bunkr(file_path): #
                    print(f"✅ Successfully downloaded {file_name}") #
                    successful = True #
                else: #
                    print(f"❌ Downloaded file is not a valid video. Removing and retrying...") #
                    os.remove(file_path) #
                    retry_count += 1 #
                    time.sleep(2) #
            except Exception as e: #
                print(f"Download attempt {retry_count + 1} failed: {e}") #
                retry_count += 1 #
                time.sleep(2) #
        return successful #
    except Exception as e: #
        print(f"An error occurred while downloading {file_name}: {e}") #
        return False #

def process_single_file_bunkr(file_url, headers, download_path): #
    try:
        file_url = re.sub(r'https?://(?:bunkr|bunkrr)\.(?:is|cr|ru|to|la|se)', 'https://bunkr.ru', file_url) #
        if '/f/' in file_url: #
            file_name = extract_filename_from_url_bunkr(file_url) #
            return download_file_bunkr(file_url, file_name, download_path, headers) #
        download_url = get_direct_download_url_bunkr(file_url, headers) #
        if download_url: #
            file_name = extract_filename_from_url_bunkr(download_url) #
            return download_file_bunkr(download_url, file_name, download_path, headers) #
        return False #
    except Exception as e: #
        print(f"Error processing single file: {e}") #
        return False #

def process_direct_cache_url_bunkr(url, headers, download_path): #
    try: #
        file_name = extract_filename_from_url_bunkr(url) #
        print(f"Downloading direct cache file: {file_name}") #
        if not any(ext in url.lower() for ext in ['.mp4', '.webm', '.mov', '.avi', '.mkv']): #
            print("⚠️ URL doesn't appear to be a video file. Checking content type...") #
            try: #
                head_response = requests.head(url, headers=headers, timeout=10) #
                content_type = head_response.headers.get('content-type', '') #
                if 'video' not in content_type and 'octet-stream' not in content_type: #
                    print(f"⚠️ Content-Type '{content_type}' doesn't appear to be video. Will verify after download.") #
            except Exception as e: #
                print(f"Could not check content type: {e}") #
        return download_file_bunkr(url, file_name, download_path, headers) #
    except Exception as e: #
        print(f"Error processing direct cache URL: {e}") #
        return False #

def download_bunkr_content_custom(url, custom_download_path): #
    try: #
        download_path = custom_download_path if custom_download_path else '/content/bunkr_downloads' #
        if not os.path.exists(download_path): #
            os.makedirs(download_path, exist_ok=True) #
            print(f"Created download directory: {download_path}") #
        else: #
            print(f"Using existing download directory: {download_path}") #
        try: #
            url_info = extract_id_from_url_bunkr(url) #
            content_type = url_info['type'] #
            content_id = url_info['id'] #
            print(f"Content type: {content_type}, ID: {content_id}") #
        except ValueError as e: #
            print(str(e)) #
            return #
        headers = { #
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36', #
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8', #
            'Accept-Language': 'en-US,en;q=0.5', #
            'Referer': 'https://bunkr.ru/', #
            'DNT': '1', #
            'Connection': 'keep-alive', #
            'Upgrade-Insecure-Requests': '1', #
            'Sec-Fetch-Dest': 'document', #
            'Sec-Fetch-Mode': 'navigate', #
            'Sec-Fetch-Site': 'same-origin', #
            'Sec-Fetch-User': '?1', #
            'Cache-Control': 'max-age=0', #
        }
        if content_type == 'direct': #
            print("Processing direct cache URL...") #
            success = process_direct_cache_url_bunkr(url, headers, download_path) #
            if success: #
                print("\nFile download completed!") #
                print(f"Video saved to: {download_path}") #
            else: #
                print("Failed to download the video file.") #
            return #
        if content_type == 'file': #
            print("Processing individual file...") #
            success = process_single_file_bunkr(url, headers, download_path) #
            if success: #
                print("\nFile download completed!") #
                print(f"Video saved to: {download_path}") #
            else: #
                print("Failed to download the video file.") #
            return #

        media_items = [] #
        domains = ['bunkr.ru', 'bunkr.is', 'bunkr.to', 'bunkr.la'] #
        api_data_found = False #
        for domain in domains: #
            if api_data_found: break #
            api_url = f"https://{domain}/api/album/{content_id}" #
            try: #
                print(f"Attempting to fetch album data from API: {api_url}") #
                api_response = requests.get(api_url, headers=headers, timeout=10) #
                if api_response.status_code == 200: #
                    try: #
                        api_data = api_response.json() #
                        if 'files' in api_data: #
                            print("Successfully retrieved album data from API") #
                            for file_item in api_data['files']: #
                                if 'name' in file_item and 'url' in file_item: #
                                    file_name = file_item['name'].lower() #
                                    if any(file_name.endswith(ext) for ext in ['.mp4', '.webm', '.mov', '.avi', '.mkv']): #
                                        media_items.append({'url': file_item['url'], 'name': file_item['name']}) #
                            if media_items: #
                                print(f"Found {len(media_items)} video files in album") #
                                api_data_found = True #
                            else: #
                                print("No video files found in album API data") #
                    except: print("Could not parse API response as JSON") #
                else: print(f"API request failed with status code {api_response.status_code}") #
            except Exception as e: print(f"Error accessing API: {e}") #

        if not media_items: #
            print("API approach failed, falling back to web scraping...") #
            for domain in domains: #
                if media_items: break #
                album_url = f"https://{domain}/a/{content_id}" #
                try: #
                    print(f"Trying album URL: {album_url}") #
                    response = requests.get(album_url, headers=headers, timeout=15) #
                    if response.status_code != 200: #
                        print(f"Error: Album page returned status code {response.status_code}") #
                        continue #
                    soup = BeautifulSoup(response.text, 'html.parser') #
                    scripts = soup.find_all('script') #
                    for script in scripts: #
                        if script.string: #
                            if 'window.__NUXT__' in script.string: #
                                try: #
                                    start_idx = script.string.find('window.__NUXT__') + len('window.__NUXT__ = ') #
                                    json_str = script.string[start_idx:] #
                                    data = json.loads(json_str) #
                                    if 'state' in data and 'albums' in data['state']: #
                                        for album_key, album_info in data['state']['albums'].items(): #
                                            if 'medias' in album_info: #
                                                for media in album_info['medias']: #
                                                    if 'streamUrl' in media: #
                                                        name = media.get('name', f"file_{media.get('id', len(media_items))}") #
                                                        if any(name.lower().endswith(ext) for ext in ['.mp4', '.webm', '.mov', '.avi', '.mkv']): #
                                                            media_items.append({'url': media['streamUrl'],'name': name}) #
                                        print(f"Found {len(media_items)} video files in NUXT data") #
                                except Exception as e: print(f"Error parsing NUXT data: {e}") #
                            elif '__INITIAL_STATE__' in script.string: #
                                try: #
                                    pattern = r'__INITIAL_STATE__\s*=\s*(\{.*?\});' #
                                    match = re.search(pattern, script.string, re.DOTALL) #
                                    if match: #
                                        state_data = json.loads(match.group(1)) #
                                        if 'album' in state_data and 'files' in state_data['album']: #
                                            for file_item in state_data['album']['files']: #
                                                if 'name' in file_item and 'url' in file_item: #
                                                    name = file_item['name'] #
                                                    if any(name.lower().endswith(ext) for ext in ['.mp4', '.webm', '.mov', '.avi', '.mkv']): #
                                                        media_items.append({'url': file_item['url'], 'name': name}) #
                                            print(f"Found {len(media_items)} video files in INITIAL_STATE data") #
                                except Exception as e: print(f"Error parsing INITIAL_STATE data: {e}") #
                            file_urls = re.findall(r'(https?://[^"\']*(?:\.mp4|\.webm|\.mov|\.avi|\.mkv))', script.string) #
                            for item_url in file_urls: #
                                name = extract_filename_from_url_bunkr(item_url) #
                                media_items.append({'url': item_url, 'name': name}) #
                    if not media_items: #
                        print("Looking for media elements in HTML...") #
                        grid_items = soup.select('.grid-item a, .media-item a, .gallery-item a') #
                        for item in grid_items: #
                            href = item.get('href') #
                            if href and ('/d/' in href or '/v/' in href): #
                                item_url = f"https://{domain}{href}" if not href.startswith('http') else href #
                                item_name = item.text.strip() or extract_filename_from_url_bunkr(href) #
                                if any(ext in href.lower() for ext in ['/v/', '.mp4', '.webm', '.mov']): #
                                    media_items.append({'url': item_url, 'name': item_name}) #
                        media_elements = soup.select('video source, a.download-link') #
                        for elem in media_elements: #
                            src = elem.get('src') or elem.get('href') #
                            if src: #
                                if not src.startswith('http'): src = f"https://{domain}{src}" #
                                name = extract_filename_from_url_bunkr(src) #
                                media_items.append({'url': src, 'name': name}) #
                        if not media_items: #
                            print("Last resort: checking all links for video content...") #
                            links = soup.find_all('a') #
                            for link_item in links: #
                                href = link_item.get('href') #
                                if href and ('/v/' in href): #
                                    item_url = f"https://{domain}{href}" if not href.startswith('http') else href #
                                    item_name = link_item.text.strip() or extract_filename_from_url_bunkr(href) #
                                    media_items.append({'url': item_url, 'name': item_name}) #
                except Exception as e: print(f"Error during web scraping with {domain}: {e}") #

        seen_urls = set() #
        unique_media_items = [] #
        for item in media_items: #
            if item['url'] not in seen_urls: #
                seen_urls.add(item['url']) #
                unique_media_items.append(item) #
        media_items = unique_media_items #

        if not media_items: #
            print("No video files found in the album or unable to parse the album page.") #
            print("Attempting direct API approach for individual files...") #
            for domain in domains: #
                if media_items: break #
                try: #
                    direct_api_url = f"https://{domain}/api/files/album/{content_id}" #
                    print(f"Trying direct API: {direct_api_url}") #
                    api_response = requests.get(direct_api_url, headers=headers, timeout=10) #
                    if api_response.status_code == 200: #
                        try: #
                            file_data = api_response.json() #
                            if isinstance(file_data, list) and len(file_data) > 0: #
                                for file_item in file_data: #
                                    if 'name' in file_item and 'url' in file_item: #
                                        name = file_item['name'] #
                                        if any(name.lower().endswith(ext) for ext in ['.mp4', '.webm', '.mov', '.avi', '.mkv']): #
                                            media_items.append({'url': file_item['url'], 'name': name}) #
                                print(f"Found {len(media_items)} video files through direct API approach") #
                        except: print("Failed to parse direct API response") #
                except Exception as e: print(f"Error with direct API approach using {domain}: {e}") #
            if not media_items: #
                 print("No video files found in the album after all attempts.") #
                 return #

        print(f"Found {len(media_items)} video files to download.") #
        downloaded_count = 0 #
        for i, item in enumerate(media_items, 1): #
            file_url = item['url'] #
            file_name = item['name'] #
            if not file_name or file_name.isspace(): file_name = extract_filename_from_url_bunkr(file_url) #
            if not any(file_name.lower().endswith(ext) for ext in ['.mp4', '.webm', '.mov', '.avi', '.mkv']): #
                print(f"Skipping {file_name}: Not a video file") #
                continue #
            if '/d/' in file_url or '/v/' in file_url or '/i/' in file_url: #
                download_url = get_direct_download_url_bunkr(file_url, headers) #
                if download_url: file_url = download_url #
                else: #
                    print(f"Skipping {file_name}: Could not get direct download URL") #
                    continue #
            print(f"\nDownloading video {i}/{len(media_items)}: {file_name}") #
            if download_file_bunkr(file_url, file_name, download_path, headers): downloaded_count += 1 #
        if downloaded_count > 0: #
            print(f"\nDownload summary: Successfully downloaded {downloaded_count} of {len(media_items)} videos.") #
            print(f"Videos saved to: {download_path}") #
        else: print("\nFailed to download any videos from this album.") #
    except Exception as e: print(f"Unexpected error during download process: {e}") #


# Function c: m3u8 Downloader
def download_with_m3u8(m3u8_link, download_location_file): #
    print(f"Starting m3u8 download for: {m3u8_link}")
    # Ensure the directory for the download_location_file exists
    file_dir = os.path.dirname(download_location_file)
    if file_dir and not os.path.exists(file_dir):
        os.makedirs(file_dir, exist_ok=True)
        print(f"Created directory for m3u8 download: {file_dir}")

    # The command uses `~/.local/bin/downloadm3u8` which might not be in PATH for subprocess
    # A more robust way is to ensure the command is found or specify its full path if known
    # For Colab, `~/.local/bin` is usually in PATH for shell commands started with `!`
    # but for subprocess, it might be different.
    command = f'~/.local/bin/downloadm3u8 -o "{download_location_file}" "{m3u8_link}"' #
    try:
        process = subprocess.Popen(command, shell=True, executable='/bin/bash', stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        stdout, stderr = process.communicate()
        if process.returncode == 0:
            print(f"m3u8 download successful! File saved as: {download_location_file}")
            print(f"Stdout: {stdout.decode()}")
        else:
            print("m3u8 download failed.")
            print(f"Stderr: {stderr.decode()}")
            print(f"Stdout: {stdout.decode()}")
    except Exception as e:
        print(f"An error occurred during m3u8 download: {e}")

# Function d: Bunkr Downloader (yt-dlp)
def download_bunkr_with_yt_dlp(bunkr_video_url, output_folder_path): #
    print(f"Starting Bunkr (yt-dlp) download for: {bunkr_video_url}")
    os.makedirs(output_folder_path, exist_ok=True) #
    cmd = [ #
        "yt-dlp",
        "--user-agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64)", #
        "--add-header", "Referer: https://bunkr.is/", #
        "--add-header", "Accept-Language: en-US,en;q=0.9", #
        "--no-check-certificate", #
        "--progress", #
        "--newline", #
        "--console-title", #
        bunkr_video_url, #
        "-o", f"{output_folder_path}/%(title)s.%(ext)s" #
    ]
    try:
        process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True) #
        for line in process.stdout: #
            print(line, end="") #
        process.wait() #
        if process.returncode == 0: #
            print(f"\n✅ yt-dlp Bunkr download complete! Check your folder: {output_folder_path}\n") #
        else: #
            print("\n❌ yt-dlp Bunkr download failed. Check the URL and try again.\n") #
    except FileNotFoundError:
        print("Error: yt-dlp command not found. Make sure it's installed and in your PATH.")
    except Exception as e:
        print(f"An error occurred during yt-dlp Bunkr download: {e}")


# --- Control Structure for Downloading ---
if not url:
    print("Error: URL is not provided. Please specify a URL in Cell 2.")
else:
    # Ensure output_path is a directory for functions that expect a directory
    # For m3u8, output_path is treated as a full file path if the user intends so,
    # otherwise, they should provide a directory.
    # For simplicity, we'll assume output_path from Cell 2 is the directory.
    # Specific file naming will be handled within each function.

    # For m3u8, if user provides a directory in output_path, we need a filename.
    # If output_path looks like a file, we use it directly for m3u8.
    m3u8_output_file = output_path
    if download_type == 'c':
        if os.path.isdir(output_path) or not '.' in os.path.basename(output_path): # Heuristic: if it's a dir or no extension
             # Try to get a filename from URL or use a default for m3u8
            parsed_m3u8_url = urlparse(url)
            m3u8_filename_from_url = os.path.basename(parsed_m3u8_url.path)
            if m3u8_filename_from_url and '.' in m3u8_filename_from_url:
                 # replace m3u8 extension with mp4 or ts
                base, ext = os.path.splitext(m3u8_filename_from_url)
                output_filename = base + ".mp4" # Defaulting to mp4
            else:
                output_filename = "downloaded_video.mp4" # Default filename
            m3u8_output_file = os.path.join(output_path.strip(os.sep), output_filename) #
        # else: m3u8_output_file is already the full path from Cell 2

    if download_type == 'a': #
        # wget -P expects a directory. output_path is already a directory path.
        download_with_wget(url, output_path.strip(os.sep))
    elif download_type == 'b': #
        # download_bunkr_content_custom expects a directory path.
        download_bunkr_content_custom(url, output_path.strip(os.sep))
    elif download_type == 'c': #
        # download_with_m3u8 expects the full output file path.
        download_with_m3u8(url, m3u8_output_file)
    elif download_type == 'd': #
        # download_bunkr_with_yt_dlp expects a directory path.
        download_bunkr_with_yt_dlp(url, output_path.strip(os.sep))
    else:
        print(f"Error: Invalid download_type '{download_type}'. Please choose from 'a', 'b', 'c', or 'd'.")

